<a href="https://colab.research.google.com/github/appleBattery/Eye-based-Verification/blob/master/Testing_model1_(Scheme_1)_on_MMU2_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import stuff

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount("/content/drive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import torch
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import numpy as np

### Define Dataset

In [ ]:
autobots = transforms.Compose([transforms.Resize((224,224)),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])

In [ ]:
# Load the mmu dataset by index from disk. but note that the 50th image is a bummer because it only got right eye data. 
class mmu2_dataset(Dataset):
  def __init__(self):
    self.path = '/content/drive/My Drive/iris detection/data/MMU 2/'
  def __len__(self):
    return 3980

  def pos_permute(self,count):
    for i in range(1,5):
        for j in range((i+1),6):
          count -= 1
          if count==0:
            return (str(i).zfill(2),str(j).zfill(2))
  
  def neg_permute(self,count,idx1):
    
    permute1 = (count//2) if (count%2)==0 else ((count//2)+1)
    permute2 = count%5 if (count%5)!=0 else 5
    permute1 = str(permute1).zfill(2); permute2 = str(permute2).zfill(2)

    if idx1<90:
      neg_idx = idx1+count
      neg_idx = neg_idx if (neg_idx!=50) else 100
      neg_idx = str(neg_idx).zfill(2) if (neg_idx<10) else str(neg_idx)
    else:
      neg_idx = idx1-count
      neg_idx = neg_idx if (neg_idx!=50) else 100
      neg_idx = str(neg_idx).zfill(2) if (neg_idx<10) else str(neg_idx)
    return (permute1, permute2, neg_idx)

  def get_file_idx(self,idx):
    if idx<=980:
      file_idx1 = (idx//20) + 1
      file_idx1 = file_idx1 if ((idx%20)!=0) else (file_idx1-1)
      file_idx2 = '01' if (((idx%20)<=10)&((idx%20)!=0)) else '02'
      file_idx1 = str(file_idx1).zfill(2) if (file_idx1<10) else str(file_idx1)
    else:
      idx += 10
      file_idx1 = (idx//20) + 1
      file_idx1 = file_idx1 if ((idx%20)!=0) else (file_idx1-1)
      file_idx2 = '01' if (((idx%20)<=10)&((idx%20)!=0)) else '02'
      file_idx1 = str(file_idx1).zfill(2) if (file_idx1<10) else str(file_idx1)
    return (file_idx1,file_idx2,idx)
      
  def __getitem__(self, idx):
    idx+=1

    if idx<=1990: # read positive image sets from disk
      file_idx1,file_idx2,idx = self.get_file_idx(idx)
      count = idx%10
      count = count if (count!=0) else 10
      
      permute1, permute2 = self.pos_permute(count)
      
      img1 = Image.open(self.path+file_idx1+file_idx2+permute1+'.bmp')
      img2 = Image.open(self.path+file_idx1+file_idx2+permute2+'.bmp')
      label = 1.
    else: # read negative image sets from disk
      file_idx1,file_idx2,idx = self.get_file_idx(idx-1990)
      count = idx%10
      count = count if (count!=0) else 10
      
      permute1,permute2,neg_idx = self.neg_permute(count,int(file_idx1))

      img1 = Image.open(self.path+file_idx1+file_idx2+permute1+'.bmp')
      img2 = Image.open(self.path+ neg_idx +file_idx2+permute2+'.bmp')
      label = 0.
      
    img1 = autobots(img1)
    img2 = autobots(img2)
    label = torch.tensor(label)

    return (img1,img2,label)

In [ ]:
mmu2 = mmu2_dataset()

In [ ]:
print(mmu2.__getitem__(1990))

('100', '02')


### Define Classifier

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # this model was working.
print(device)
class iris_classifier(nn.Module):
  def __init__(self):
    super(iris_classifier, self).__init__()
    self.resNet50 = models.resnet50(pretrained=True)
    # Here you get the bottleneck/feature extractor
    modules=list(self.resNet50.children())[:-1]
    self.resNet50 = torch.nn.Sequential(*modules)

    self.resNet50.flat = torch.nn.Flatten()
    
    for param in self.resNet50.parameters():
      param.requires_grad = False # freeze all layers of the convnet
    
    self.fc1 = nn.Linear(2048,1024)
    self.fc2 = nn.Linear(1024,128)
    self.fc3 = nn.Linear(256,16)
    self.fc4 = nn.Linear(16,1)

    # Set your own forward pass
  def forward(self, img1, img2):
    self.resNet50.eval()
    out1 = self.resNet50(img1)
    out2 = self.resNet50(img2)
    
    out1 = F.relu(self.fc1(out1))
    out2 = F.relu(self.fc1(out2))
    out1 = F.relu(self.fc2(out1))
    out2 = F.relu(self.fc2(out2))

    out = torch.cat((out1,out2),1)

    out = F.relu(self.fc3(out))
    out = self.fc4(out)

    return out

classifier = iris_classifier()
classifier = classifier.to(device)
print(classifier)

cuda:0


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth



iris_classifier(
  (resNet50): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): C

### Load saved model

In [ ]:
model0 = iris_classifier()
model0.load_state_dict(torch.load('/content/drive/My Drive/iris detection/data/model0_100.pth'))

<All keys matched successfully>

### Testing this dataset on previously trained model:

In [ ]:
train_size = int(0.7 * len(mmu2))
test_size = len(mmu2) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(mmu2, [train_size, test_size])

In [ ]:
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64,shuffle=True, num_workers = 16)
model0 = model0.to(device)

In [ ]:
acc=0;
for data in testloader:
  img1_batch, img2_batch, labels = data
  img1_batch = img1_batch.to(device); img2_batch = img2_batch.to(device);
  img1_batch = img1_batch.type(torch.cuda.FloatTensor); img2_batch = img2_batch.type(torch.cuda.FloatTensor)

  out = model0(img1_batch,img2_batch)
  out = torch.sigmoid(out.cpu())
  predict = out.detach().numpy()
  labels = labels.numpy()
  predict[predict>=0.5] = 1.
  predict[predict<0.5] = 0.
  predict = predict.squeeze()
  acc += np.sum(predict==labels)

acc = acc/(3980*0.3)
print(acc)

0.21407035175879396
